In [5]:
import re
import pandas as pd
import numpy as np
import os

f = open('./Data/sentences_with_roles_and_relations.txt', encoding = "ISO-8859-1")
f_data = []
for line in f.readlines():
    line = line[:-1] # Remove linebreak
    f_data.append(line.split('||'))
f.close()

rows = []
for l in f_data:
    if l[1] not in ['NONE', 'TREATONLY', 'DISONLY', 'TO_SEE', 'VAGUE', 'TREAT_NO_FOR_DIS']:
        sent = ' '.join(l[0].split())
        dis_re = re.compile('<DIS.*>(.*)</DIS.*>')
        disease = dis_re.search(sent).group(1)
        treat_re = re.compile('<TREAT.*>(.*)</TREAT.*>')
        treat = treat_re.search(sent).group(1)
        
        sent = re.sub(r'<.*?> ', '', sent).strip()
        # Handles sentences ending with <*> structure
        sent = re.sub(r'<.*?>', '', sent)
        
        rows.append([sent, l[1], treat.strip(), disease.strip()])

biotext_df = pd.DataFrame(data=rows, columns=['sentence', 'relation', 'term1', 'term2'])

In [6]:
biotext_df.relation.value_counts()

TREAT_FOR_DIS    830
PREVENT           63
SIDE_EFF          30
Name: relation, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

train_df, val_df, train_labels, val_labels = train_test_split(
    biotext_df,
    biotext_df['relation'],
    test_size=0.4,
    stratify = biotext_df['relation'],
    random_state = 42
)

In [8]:
# Define our numeric labels as integers
ABSTAIN = -1
TREAT_FOR_DIS = 0
PREVENT     = 1
SIDE_EFF = 2


def map_labels(x):
    """Map string labels to integers"""
    if x == 'TREAT_FOR_DIS':
        return TREAT_FOR_DIS
    elif x == 'PREVENT':
        return PREVENT
    elif x == 'SIDE_EFF':
        return SIDE_EFF


val_labels  =  val_labels.apply(map_labels, convert_dtype=True)


In [9]:
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import labeling_function
from snorkel.labeling import LFAnalysis
from snorkel.labeling.model  import MajorityLabelVoter
from snorkel.labeling.model import LabelModel
from snorkel.labeling import LabelingFunction
from snorkel.labeling import filter_unlabeled_dataframe

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [11]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer()

In [12]:
mapping_pos_label = {"JJ":'a', 
 "RB":'r', 
 "NN": 'n', 
 "VB":'v'}

In [13]:
def get_pos_label(w, postag, mapping_pos_label):
#     print(w,"--->",postag)
    for k, v in mapping_pos_label.items():
        if postag.startswith(k):
            return v
    return "n"

In [14]:
import re
def preprocess_text(text):
    text = text.lower()
    text = " ".join([lemmatizer.lemmatize(w, 
                                  pos= get_pos_label(w, 
                                                     pos_w,
                                                     mapping_pos_label))\
                     for w, pos_w in pos_tag(text.split()) \
                     if w not in list(set(stopwords.words('english')))])
    text = re.sub(r'\d+', '', text)
    return text

In [15]:
cv = CountVectorizer(preprocessor = preprocess_text,
               ngram_range = (1,3),
               min_df = 0.01)

In [16]:
count_mat = cv.fit_transform(biotext_df[biotext_df.relation.isin(["SIDE_EFF"])].sentence)
count_df = pd.DataFrame(count_mat.todense(), columns=cv.get_feature_names())

In [17]:
count_df = count_df.sum().reset_index()
count_df.columns = ["word","val"]
count_df = count_df.sort_values('val', ascending = False)

In [18]:
import plotly.express as px
fig = px.pie(count_df.head(20), values='val', names='word', title="Top Words for 'SIDE_EFF' Text")
fig.show()

In [19]:
treatment_keywords = ['treatment', 'therapy','effective','treat', "reduce"]
prevent_keywords = ["protect", "prevent", "inhibit", "block", "control", 'effect']
side_effect_keywords = ["risk","follow", "associate", "toxic"]

In [20]:
# import nltk 
# from nltk.corpus import wordnet as wn
# synonyms = [str(l.name()).replace("_"," ") for word in prevent_keywords for syn in wn.synsets(word) for l in syn.lemmas()] 

In [21]:
# synonyms = list(np.unique(synonyms))

In [22]:
# from nltk.sentiment import SentimentIntensityAnalyzer
# sia = SentimentIntensityAnalyzer()
# sia.polarity_scores("There was also a slightly higher rate of intracranial hemorrhage with the double-bolus method .")

In [23]:
import nltk
import spacy
import scispacy
import networkx as nx

from scispacy.linking import EntityLinker
nlp = spacy.load('en_core_sci_lg')

In [24]:
linker = EntityLinker(resolve_abbreviations=False, name="umls") # keeping default thresholds for match percentage.
nlp.add_pipe(linker)

C:\Users\bansa\anaconda3\envs\snorkel\lib\site-packages\sklearn\base.py:306: UserWarning:

Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.

C:\Users\bansa\anaconda3\envs\snorkel\lib\site-packages\sklearn\base.py:306: UserWarning:

Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.



In [25]:
def remerge_sent(sent):
    i = 0
    while i < len(sent)-1:
        tok = sent[i]
        if not tok.whitespace_:
            ntok = sent[i+1]
            # in-place operation.
            sent.merge(tok.idx, ntok.idx+len(ntok))
        i += 1
    return sent

In [26]:
#     print([G.node[n]['text'] \
#            for i,n in enumerate(shortest_path) \
#            if i>0 and i<len(shortest_path)-1])
#     print([G.node[n]['text'] \
#            for shortest_path in shortest_path_list \
#            for i,n in enumerate(shortest_path) \
#            if i>0 and i<len(shortest_path)-1])

#     if i == 4:
#         spacy.displacy.serve(doc, style="dep", page="true")
#         break

# shortest_path = nx.shortest_path(G, source = entity1_idx, target = entity2_idx)

In [21]:
sdp_list = {'PREVENT': [],
           'SIDE_EFF': [],
           'TREAT_FOR_DIS': []}

In [22]:
for KEY in sdp_list.keys():
    for i,row in biotext_df[biotext_df.relation.isin([KEY])].iterrows():    
        # Entities to find SDP between
        entity1 = row["term1"].replace(" ","|").replace("`","")
        entity2 = row["term2"].replace(" ","|").replace("`","")

        # Adjusting for Space
        new_sentence = row["sentence"].replace(row["term1"], entity1)
        new_sentence = new_sentence.replace(row["term2"], entity2)

        # Spacy Pipeline 
        doc = nlp(new_sentence)
        doc = remerge_sent(doc)


        entity1_idx = [token.i for token in doc if token.text in [entity1]][0]
        entity2_idx = [token.i for token in doc if token.text in [entity2]][0]

        # Load Networkx Graph
        G = nx.Graph()

        # Load spacy's dependency tree into a networkx graph
        edges = []
        for token in doc:
            for child in token.children:
                G.add_nodes_from([(token.i, {"pos": token.pos_, 
                                             "text": token.text}),
                                  (child.i, {"pos": child.pos_, 
                                             "text": child.text})])
                edges.append((token.i,
                              child.i))

        # Addding Edges
        G.add_edges_from(edges)

        meta_info = {}
        meta_info["entity1"] = entity1
        meta_info["entity2"] = entity2
        meta_info["entity1_idx"] = entity1_idx
        meta_info["entity2_idx"] = entity2_idx
        meta_info["graph_object"] = G


        shortest_path_list = nx.all_shortest_paths(G, source = entity1_idx, target = entity2_idx)

        meta_info["word_list"] = [(G.node[n]['text'], G.node[n]['pos']) \
                                  for shortest_path in shortest_path_list \
                                  for i,n in enumerate(shortest_path) \
                                  if i>0 and i<len(shortest_path)-1]

        sdp_list[KEY].append(meta_info)

C:\Users\bansa\anaconda3\envs\snorkel\lib\site-packages\scispacy\candidate_generation.py:284: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray

C:\Users\bansa\anaconda3\envs\snorkel\lib\site-packages\scispacy\candidate_generation.py:285: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray

C:\Users\bansa\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:8: DeprecationWarning:

[W013] As of v2.1.0, Doc.merge is deprecated. Please use the more efficient and less error-prone Doc.retokenize context manager instead.



In [27]:
mapping_pos_label_spacy = {"ADJ":'a', 
 "ADV":'r', 
 "NOUN": 'n', 
 "VERB":'v'}

In [24]:
lemmatized_list = [[lemmatizer.lemmatize(word[0].lower(), 
                                         get_pos_label(word[0], 
                                                       word[1], 
                                                       mapping_pos_label_spacy)) \
                    for word in val['word_list']] \
                   for val in sdp_list["TREAT_FOR_DIS"] \
                   if len(val['word_list']) > 0]

In [25]:
#take individual wordlists from the lemmatized tokens
# create 1 to 3 gram tokens
# Find Frequency

In [28]:
def get_top_words(lemmatized_list, n):
    """
    Show Top 'n' words
    """
    count_df = pd.Series([" ".join(word_phrase) \
                          for word_list in lemmatized_list \
                          for i in range(1,4) \
                          for word_phrase in nltk.ngrams(word_list, i)]).value_counts().reset_index()
    count_df.columns = ["word","counts"]
    
    count_df = count_df[count_df.counts > 1]
    for i,row in count_df.head(n).iterrows():
        print(row["word"] ,"---->", row["counts"])

In [29]:
get_top_words(lemmatized_list, 20)

### Distance Supervision

In [63]:
import spacy
import scispacy
import networkx as nx

from scispacy.linking import EntityLinker
nlp = spacy.load('en_core_sci_lg')

In [64]:
linker = EntityLinker(resolve_abbreviations=False, name="umls") # keeping default thresholds for match percentage.
nlp.add_pipe(linker)

C:\Users\bansa\anaconda3\envs\snorkel\lib\site-packages\sklearn\base.py:306: UserWarning:

Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.

C:\Users\bansa\anaconda3\envs\snorkel\lib\site-packages\sklearn\base.py:306: UserWarning:

Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.



In [29]:
# UMLs provides a class name to each of its TXXX identifier, TXXX is code for parents for each of the CUI numbers a unique concept
# identifier used by UMLs Kb

# To obtain this file please login to https://www.nlm.nih.gov/research/umls/index.html
# Shared in Github Repo of the book :)
type2namemap = pd.read_csv("SRDEF", sep ="|", header = None)
type2namemap = type2namemap.iloc[:,:3]
type2namemap.columns = ["ClassType","TypeID","TypeName"]
typenamemap = {row["TypeID"]:row["TypeName"] for i,row in type2namemap.iterrows()}

In [115]:
KEY = "TREAT_FOR_DIS"

In [116]:
umls_concept_extracted = [[umls_ent for entity in doc.ents for umls_ent in entity._.umls_ents] for doc in nlp.pipe(biotext_df[biotext_df.relation.isin([KEY])].sentence.tolist())]
umls_concept_cui = [linker.kb.cui_to_entity[concept[0]] for concepts in umls_concept_extracted for concept in concepts]
# Capturing all the information shared from the UMLS DB in a dataframe
umls_concept_df = pd.DataFrame(umls_concept_cui)
concept_df = pd.Series([typenamemap[typeid] for types in umls_concept_df.types for typeid in types]).value_counts().reset_index()
concept_df.columns = ["concept","count"]

umls_concept_df["Name"] = pd.Series([[typenamemap[typeid] for typeid in types] for types in umls_concept_df.types])


In [119]:
# concept_df.to_csv(KEY+".csv")
# umls_concept_df.to_csv(KEY+"vv.csv")

### Snorkelling

#### Regex

In [30]:
treatment_keywords = ['treatment', 'therapy','effective','treat', "reduce"]
prevent_keywords = ["protect", "prevent", "inhibit", "block", "control", 'effect']
side_effect_keywords = ["risk","follow", "associate", "toxic"]

In [31]:
@labeling_function()
def sent_contains_TREAT_FOR_DIS(x):
    text = x.sentence.lower()
    lemmatized_word = [lemmatizer.lemmatize(w, 
                                  pos= get_pos_label(w, 
                                                     pos_w,
                                                     mapping_pos_label))\
                     for w, pos_w in pos_tag(text.split()) \
                     if w not in list(set(stopwords.words('english')))]
    return TREAT_FOR_DIS if any([ True if key in lemmatized_word else False for key in treatment_keywords]) else ABSTAIN

@labeling_function()
def sent_contains_SIDE_EFF(x):
    text = x.sentence.lower()
    lemmatized_word = [lemmatizer.lemmatize(w, 
                                  pos= get_pos_label(w, 
                                                     pos_w,
                                                     mapping_pos_label))\
                     for w, pos_w in pos_tag(text.split()) \
                     if w not in list(set(stopwords.words('english')))]
    return SIDE_EFF if any([ True if key in lemmatized_word else False for key in side_effect_keywords]) else ABSTAIN

@labeling_function()
def sent_contains_PREVENT(x):
    text = x.sentence.lower()
    lemmatized_word = [lemmatizer.lemmatize(w, 
                                  pos= get_pos_label(w, 
                                                     pos_w,
                                                     mapping_pos_label))\
                     for w, pos_w in pos_tag(text.split()) \
                     if w not in list(set(stopwords.words('english')))]
    return PREVENT if any([ True if key in lemmatized_word else False for key in prevent_keywords]) else ABSTAIN

#### Syntactic

In [32]:
from snorkel.preprocess import preprocessor

@preprocessor(memoize = True)
def get_syntactic_info(x):
    
    # Entities to find SDP between
    entity1 = x.term1.replace(" ","|").replace("`","")
    entity2 = x.term2.replace(" ","|").replace("`","")

    # Adjusting for Space
    new_sentence = x.sentence.replace(x.term1, entity1)
    new_sentence = new_sentence.replace(x.term2, entity2)

    # Spacy Pipeline 
    doc = nlp(new_sentence)
    doc = remerge_sent(doc)


    entity1_idx = [token.i for token in doc if token.text in [entity1]][0]
    entity2_idx = [token.i for token in doc if token.text in [entity2]][0]

    # Load Networkx Graph
    G = nx.Graph()

    # Load spacy's dependency tree into a networkx graph
    edges = []
    for token in doc:
        for child in token.children:
            G.add_nodes_from([(token.i, {"pos": token.pos_, 
                                         "text": token.text}),
                              (child.i, {"pos": child.pos_, 
                                         "text": child.text})])
            edges.append((token.i,
                          child.i))

    # Addding Edges
    G.add_edges_from(edges)

    shortest_path_list = nx.all_shortest_paths(G, source = entity1_idx, target = entity2_idx)

    word_list = [(G.node[n]['text'], G.node[n]['pos']) \
                              for shortest_path in shortest_path_list \
                              for i,n in enumerate(shortest_path) \
                              if i>0 and i<len(shortest_path)-1]
    
    lemmatized_list = [lemmatizer.lemmatize(word[0].lower(), 
                                         get_pos_label(word[0], 
                                                       word[1], 
                                                       mapping_pos_label_spacy)) \
                        for word in word_list]

    
    x.sdp_word = lemmatized_list
    return x

In [33]:
treatment_sdp_keywords = ['patient', 'use','trial','management', "study", "show", "improve"]
prevent_sdp_keywords = ["reduce", "vaccine", "incidence", "stability"]
side_effect_sdp_keywords = ["rate","case", "administration", "complication", "develop"]

In [34]:
@labeling_function(pre=[get_syntactic_info])
def sent_sdp_TREAT_FOR_DIS(x):
    return TREAT_FOR_DIS if any([True if key in x.sdp_word else False for key in treatment_sdp_keywords]) else ABSTAIN

@labeling_function(pre=[get_syntactic_info])
def sent_sdp_SIDE_EFF(x):
    return SIDE_EFF if any([True if key in x.sdp_word else False for key in side_effect_sdp_keywords]) else ABSTAIN

@labeling_function(pre=[get_syntactic_info])
def sent_sdp_PREVENT(x):
    return PREVENT if any([True if key in x.sdp_word else False for key in prevent_sdp_keywords]) else ABSTAIN

#### UMLs

In [39]:
@preprocessor(memoize = True)
def get_umls_concepts(x):
    
    umls_concept_extracted = [[umls_ent for entity in doc.ents for umls_ent in entity._.umls_ents] for doc in nlp.pipe([x.sentence])]
    
    try:
        umls_concept_cui = [linker.kb.cui_to_entity[concept[0]] for concepts in umls_concept_extracted for concept in concepts]
        # Capturing all the information shared from the UMLS DB in a dataframe
        umls_concept_df = pd.DataFrame(umls_concept_cui)
        concept_df = pd.Series([typenamemap[typeid] for types in umls_concept_df.types for typeid in types]).value_counts().reset_index()
        concept_df.columns = ["concept","count"]

        x["umls_concepts"] = concept_df.concept.tolist()
    except Exception as e:
        x["umls_concepts"] = []
    
    return x

In [52]:
treatment_umls_concepts = ['Therapeutic or Preventive Procedure', 
                           'Intellectual Product',
                           'Qualitative Concept',
                           'Patient or Disabled Group', 
                           "Temporal Concept", 
                           "Health Care Activity"]

prevent_umls_concepts = ["Immunologic Factor", 
                         "Idea or Concept", 
                         "Occupational Activity"]

side_effect_umls_concepts = ["Sign or Symptom",
                             "Injury or Poisoning", 
                             "Body Part, Organ, or Organ Component", 
                             "Pathologic Function"]

In [53]:
@labeling_function(pre=[get_umls_concepts])
def sent_umls_TREAT_FOR_DIS(x):
    return TREAT_FOR_DIS if any([True if key in x.umls_concepts else False for key in treatment_umls_concepts]) else ABSTAIN

@labeling_function(pre=[get_umls_concepts])
def sent_umls_SIDE_EFF(x):
    return SIDE_EFF if any([True if key in x.umls_concepts else False for key in side_effect_umls_concepts]) else ABSTAIN

@labeling_function(pre=[get_umls_concepts])
def sent_umls_PREVENT(x):
    return PREVENT if any([True if key in x.umls_concepts else False for key in prevent_umls_concepts]) else ABSTAIN

In [54]:
lfs = [sent_contains_TREAT_FOR_DIS, sent_contains_SIDE_EFF, sent_contains_PREVENT,
      sent_sdp_TREAT_FOR_DIS, sent_sdp_SIDE_EFF, sent_sdp_PREVENT,
      sent_umls_TREAT_FOR_DIS, sent_umls_SIDE_EFF, sent_umls_PREVENT]

# Instantiate our LF applier with our list of LabelFunctions (just one for now)
applier = PandasLFApplier(lfs=lfs)

# Apply the LFs to the data to generate a list of labels
L_train = applier.apply(df=train_df)
L_dev   = applier.apply(df=val_df)

# Run a label function analysis on the results, to describe their output against the labeled development data
LFAnalysis(L=L_dev, lfs=lfs).lf_summary(val_labels.values)

100%|████████████████████████████████████████████████████████████████████████████████| 370/370 [00:13<00:00, 26.98it/s]


j Polarity  Coverage  Overlaps  Conflicts  \
sent_contains_TREAT_FOR_DIS  0      [0]  0.505405  0.502703   0.305405   
sent_contains_SIDE_EFF       1      [2]  0.051351  0.051351   0.051351   
sent_contains_PREVENT        2      [1]  0.113514  0.113514   0.113514   
sent_sdp_TREAT_FOR_DIS       3      [0]  0.329730  0.324324   0.229730   
sent_sdp_SIDE_EFF            4      [2]  0.040541  0.040541   0.040541   
sent_sdp_PREVENT             5      [1]  0.024324  0.024324   0.024324   
sent_umls_TREAT_FOR_DIS      6      [0]  0.943243  0.875676   0.627027   
sent_umls_SIDE_EFF           7      [2]  0.294595  0.291892   0.291892   
sent_umls_PREVENT            8      [1]  0.454054  0.443243   0.443243   

                             Correct  Incorrect  Emp. Acc.  
sent_contains_TREAT_FOR_DIS      177         10   0.946524  
sent_contains_SIDE_EFF             4         15   0.210526  
sent_contains_PREVENT             11         31   0.261905  
sent_sdp_TREAT_FOR_DIS           113          9   0.926230  
sent_sdp_SIDE_EFF                  5         10   0.333333  
sent_sdp_PREVENT                   5          4   0.555556  
sent_umls_TREAT_FOR_DIS          316         33   0.905444  
sent_umls_SIDE_EFF                 7        102   0.064220  
sent_umls_PREVENT                 21        147   0.125000

In [58]:
from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter(cardinality = 3)
preds_train = majority_model.predict(L=L_train)

In [60]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=3, verbose=True)
label_model.fit(L_train=L_train, n_epochs=100, seed=42)

In [63]:
majority_acc = majority_model.score(L=L_dev, Y=val_labels, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_dev, Y=val_labels, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

Majority Vote Accuracy:   80.8%
Label Model Accuracy:     87.6%
